# Documentation of Economic Analysis behind Simulation Engine - Part 2
We want to measure what happens when parts of the economy shut down. To do so, we need to model how we "switch off" a sector from the matrix (partially or totally). In this notebook, we provide a set of functions that can exactly do that. Ideally, the goal is to use them with the EMA workbench or a similar simulation workbench to produce multiple "what-if" scenarios through Monte Carlo simulation. For the purposes of this notebook, they are useful to illustrate the different types of shocks that we can apply to the system.

To do so, we are going to propagate a change in the production of one sector throughout the rest of the network (either upstream or downstream) using the Leontief inverse matrix. The way we will shock a sector is by shocking its final output vector, which we have called x. This will give us a vector of shocks whose entries will be the first-order change in output of all sectors (i.e. zero in all but for the shocked sector). We will then take the dot product of the Leontief inverse and the shocks vector, which will give us all the direct and indirect changes in all sectors. Finally, we'll add it to the original outputs vector, getting the new "state of the world" after the shock.

The functions come in two flavours: the first function shocks the total demand for a sector (i.e. it answers the question "what happens when one sector reduces production by $\Delta\%$" and is simpler to work with); the second is a bit more granular, and takes into account the fact that the final demand (matrix Y) has different components (think of households' consumption, public consumption, etc.). Hence, the second function allows us to shock specific components separately (i.e. it can answer the question "what happens when the government reduces its consumption from sector X by $\Delta\%$"). The third function wraps the first two. Since it makes it a bit more complex, we present the three functions as separate pieces; that way the end user can decide which one to use in their simulations.

All three functions can take two different types of shocks: supply-side and demand-side. The difference between a supply-side and a demand-side shock is the direction of the propagation of the shock: supply side shocks are upstream or, in other words, they come from further up the supply chain that feeds into the shocked sector. On the other hand, demand-side shocks have an impact downstream, meaning that they come from the sectors that consume the shocked sector's output.





**Table of contents**
* [Shock total demand](#total_shock)
* [Shock demand components](#components_shock)
* [Function to shock anything](#shock_anything)

**Inputs**
- Matrices L, x, Z, Y from notebook *Documentation of Economic Analysis behind Simulation Engine - Part 1*

In [ ]:
# Imports and path
import pandas as pd
import numpy as np
import random


<a id='total_shock'></a>
### Shock total demand

In [14]:
def shock_demand(L, x, country, sector, shock = -0.5, downstream = False):
    
    '''
    This function transmits a shock to one sector of a country's economy throught a network of sectors and countries 
    using their input-output linkages as defined by a Leontief inverse matrix L.
    
    Arguments:
    - L: Leontief inverse matrix. Entry (i, j) summarises all direct and indirect connections between (country, sector) i and 
    (country, sector) j. Dimensions must be (N x N), N being the number of country-sector combinations.
    - x: column vector with total outputs by country and sector. Dimesions must be (N x 1).
    - country: part of multiindex of L and x. The country of the sector that will be shocked.
    - sector: part of multiindex of L and x. The sector to be shocked.
    - shock: relative change in the output of a sector. By default, it's -0.5, i.e. a 50% decrease in output.
    - downstream: whether we want to propagate the shock upstream (supply shock) or downstream (demand shock). Default is False, i.e. upstream shock
    '''
    
    # Construct a shocks vector with all zeros but for the (country, sector) entry that we want to shock
    shock_vector = pd.DataFrame(np.zeros(x.shape), index = x.index, columns = ['delta'])
    shock_vector.loc[(country, sector)] = shock
    
    # Compute changes in (country, sector) X 
    x_prime = x.copy()
    x_prime['indout'] = shock_vector['delta'] * x['indout']
    
    # Compute the inner product of X' and Leontief inverse to get all changes
    if downstream == True:
        L = np.transpose(L) # By transposing the leontief inverse, we get the effect of the shock on the sectors downstream of the shocked sector
    
    output_change = x.copy()
    output_change['indout'] = np.dot(L, x_prime)
    
    # New X is original X plus changes
    new_output = x.copy()
    new_output = x + output_change
    new_output[new_output['indout'] < 0] = 0 # output can't be negative, 0 at most
    
    return new_output

In [15]:
# Test on sector 01T03 in Argentina 
## Shock matrix
x_new = shock_demand(L, x, country = 'ARG', sector = '01T03', shock = -0.9)
x_new['before'] = x['indout']
x_new['Relative change, %'] = 100 * (x_new['indout'] - x_new['before'])/ x_new['before']
x_new.head(10)

indout        before  Relative change, %
region sector                                                           
ARG    01T03              0.000000e+00  8.834342e+04         -100.000000
       05T06              2.789205e+04  3.053964e+04           -8.669364
       09                 3.685767e+03  4.231662e+03          -12.900237
       10T12              9.091610e+04  9.322885e+04           -2.480723
       13T15              3.287748e+04  3.331857e+04           -1.323846
...                                ...           ...                 ...
TUR    All non-essential  2.177606e+05  2.177646e+05           -0.001827
TWN    All non-essential  1.552742e+05  1.552781e+05           -0.002558
USA    All non-essential  4.661414e+06  4.661483e+06           -0.001476
VNM    All non-essential  5.347955e+04  5.348071e+04           -0.002175
ZAF    All non-essential  1.011204e+05  1.011248e+05           -0.004345

[2015 rows x 3 columns]

<a id='components_shock'></a>
### Shock separate components of final demand

In [16]:
def shock_demand_components(L, Z, Y, x, country, sector, demand_component, shock = -0.5, downstream = False):

    '''
    This function transmits a shock to a component of the demand for one sector of a country's economy throught a network of 
    sectors and countries using their input-output linkages as defined by a Leontief inverse matrix L.
    
    Arguments:
    - L: Leontief inverse matrix. Entry (i, j) summarises all direct and indirect connections between (country, sector) i and 
    (country, sector) j. Dimensions must be (N x N), N being the number of country-sector combinations.
    - Z: Input-Output matrix of output flows between sectors. Dimensions must be (N x N).
    - Y: Aggregate demand by sector. Dimensions must be (N x M), M being the number of country-demand components combinations.
    - x: column vector with total outputs by country and sector. Dimesions must be (N x 1).
    - country: part of multiindex of L and x. The country of the sector that will be shocked.
    - sector: part of multiindex of L and x. The sector to be shocked.
    - demand_component: component of sector's aggregate demand to be shocked. Possible values are:
        - 'HFCE'
        - 'NPISH'
        - 'GGFC'
        - 'GFCF'
        - 'INVNT'
        - 'P33'
    - shock: relative change in the output of a sector. By default, it's -0.5, i.e. a 50% decrease in demand.
    - downstream: whether we want to propagate the shock upstream (supply shock) or downstream (demand shock). Default is False, i.e. upstream shock
    '''
        
    # Construct a shocks vector with all zeros but for the (country, sector) entry that we want to shock
    shock_matrix = pd.DataFrame(np.zeros(Y.shape), index = Y.index, columns = Y.columns)
    shock_matrix.loc[(country, sector), (country, demand_component)] = shock
    
    # Compute changes in Y 
    Y_prime = Y.copy()
    Y_prime = Y_prime.multiply(shock_matrix)
    
    Z_prime = pd.DataFrame(np.zeros(Z.shape), index = Z.index, columns = Z.columns) # Pseudo-Z matrix will all zeros (as nothing changes in Z)
    
    x_prime = pymrio.calc_x(Z_prime, Y_prime) # Collapse Y_prime to get a column vector of the shape of x
    
    # Compute the inner product of X' and Leontief inverse to get all changes
    if downstream == True:
        L = np.transpose(L) # By transposing the leontief inverse, we get the effect of the shock on the sectors downstream of the shocked sector
    
    output_change = x.copy()
    output_change['indout'] = np.dot(L, x_prime)
    
    # New X is original X plus changes
    new_output = x.copy()
    new_output = x + output_change
    new_output[new_output['indout'] < 0] = 0 # output can't be negative, 0 at most

    return new_output


In [17]:
# Test on household's (HFCE) demand for sector 01T03 in Argentina 
x_new = shock_demand_components(L, Z, Y, x, country = 'ARG', sector = '01T03', demand_component = 'HFCE', shock = -0.9)
x_new['before'] = x['indout']
x_new['Relative change, %'] = 100 * (x_new['indout'] - x_new['before'])/ x_new['before']
round(x_new.head(10), 3)

indout      before  Relative change, %
region sector                                            
ARG    01T03    73907.445   88343.425             -16.341
       05T06    30134.937   30539.638              -1.325
       09        4148.218    4231.662              -1.972
       10T12    92875.335   93228.853              -0.379
       13T15    33251.146   33318.569              -0.202
       16        7807.276    7867.541              -0.766
       17T18    15063.634   15134.832              -0.470
       19       36301.628   36977.334              -1.827
       20T21    26740.455   27134.519              -1.452
       22       18448.720   18599.630              -0.811
       23        9397.744    9455.702              -0.613
       26        6154.082    6159.404              -0.086
       27        7774.596    7802.912              -0.363
       28       10043.383   10072.038              -0.284
       31T33     5414.102    5420.127              -0.111
       35T39    24193.107   24339.703              -0.602
       41T43    44605.335   44613.024              -0.017
       45T47   101714.291  102535.500              -0.801
       49T53    55036.113   55199.990              -0.297
       55T56    27298.075   27322.160              -0.088
       58T60    18623.902   18673.291              -0.264
       61       15815.747   15831.988              -0.103
       62T63     5056.206    5066.557              -0.204
       64T66    25441.300   25614.122              -0.675
       69T82    43683.949   43936.202              -0.574
       84       30523.161   30524.214              -0.003
       85       30303.576   30304.134              -0.002
       86T88    24004.147   24039.270              -0.146
       90T96    26535.401   26557.417              -0.083
       97T98     7504.337    7504.337               0.000

<a id='shock_anything'></a>
### Function to shock anything

In [18]:
def shock_anything(L, x, Z = None, Y = None, split_demand = False, 
                    country = None, sector = None, demand_component = None, 
                    shock = -0.5, downstream = False):

    '''
    This function transmits a shock to the demand for one sector of a country's economy through a network of sectors and countries using their 
    input-output linkages as defined by a Leontief inverse matrix L. The shock can be to the whole aggregate demand or to a particular component 
    (see below), depending on the value of split_demand.
    
    Arguments:
    - L: Leontief inverse matrix. Entry (i, j) summarises all direct and indirect connections between (country, sector) i and 
    (country, sector) j. Dimensions must be (N x N), N being the number of country-sector combinations.
    - x: column vector with total outputs by country and sector. Dimesions must be (N x 1).
    - Z: Input-Output matrix of output flows between sectors. Dimensions must be (N x N). Only applies when split_demand = True, so by default Z = None.
    - Y: Aggregate demand by sector. Dimensions must be (N x M), M being the number of country-demand components combinations. Only applies when split_demand = True, so by default Y = None.
    - split_demand: Boolean. Indicate if the shock is to the whole demand or just a part of it.
    - country: part of multiindex of L and x. The country of the sector that will be shocked.
    - sector: part of multiindex of L and x. The sector to be shocked.
    - demand_component: component of sector's aggregate demand to be shocked. Possible values are:
        - 'HFCE'
        - 'NPISH'
        - 'GGFC'
        - 'GFCF'
        - 'INVNT'
        - 'P33'
    - shock: relative change in the output of a sector. By default, it's -0.5, i.e. a 50% decrease in demand.
    - downstream: whether we want to propagate the shock upstream (supply shock) or downstream (demand shock). Default is False, i.e. upstream shock
    '''
    
    if (split_demand == False) & (country != None)  & (sector != None):
        
        # Construct a shocks vector with all zeros but for the (country, sector) entry that we want to shock
        shock_vector = pd.DataFrame(np.zeros(x.shape), index = x.index, columns = ['delta'])
        shock_vector.loc[(country, sector)] = shock

        # Compute changes in (country, sector) X 
        x_prime = x.copy()
        x_prime['indout'] = shock_vector['delta'] * x['indout']

    
    elif (split_demand == True) & (country != None)  & (sector != None)  & (demand_component != None):
        
        # Construct a shocks vector with all zeros but for the (country, sector) entry that we want to shock
        shock_matrix = pd.DataFrame(np.zeros(Y.shape), index = Y.index, columns = Y.columns)
        shock_matrix.loc[(country, sector), (country, demand_component)] = shock

        # Compute changes in Y 
        Y_prime = Y.copy()
        Y_prime = Y_prime.multiply(shock_matrix)

        Z_prime = pd.DataFrame(np.zeros(Z.shape), index = Z.index, columns = Z.columns) # Pseudo-Z matrix will all zeros (as nothing changes in Z)

        x_prime = pymrio.calc_x(Z_prime, Y_prime) # Collapse Y_prime to get a column vector of the shape of x
    
    
    # Compute the inner product of X' and Leontief inverse to get all changes
    if downstream == True:
        L = np.transpose(L) # By transposing the leontief inverse, we get the effect of the shock on the sectors downstream of the shocked sector
    
    output_change = x.copy()
    output_change['indout'] = np.dot(L, x_prime)
    
    # New X is original X plus changes
    new_output = x.copy()
    new_output = x + output_change
    new_output[new_output['indout'] < 0] = 0 + np.random.uniform(0, 0.01) # output can't be negative, 0 at most plus a small number for simulation

    return new_output

In [19]:
# Test on household's (HFCE) demand for sector 01T03 in Argentina - split_demand = False (implicit as it's the default value)
x_new = shock_anything(L, x, Z, Y, country = 'ARG', sector = '01T03', demand_component = 'HFCE', shock = -0.9)

x_new['before'] = x['indout']
x_new['Relative change, %'] = 100 * (x_new['indout'] - x_new['before'])/ x_new['before']
round(x_new.head(10), 3)

indout     before  Relative change, %
region sector                                          
ARG    01T03       0.006  88343.425            -100.000
       05T06   27892.045  30539.638              -8.669
       09       3685.767   4231.662             -12.900
       10T12   90916.104  93228.853              -2.481
       13T15   32877.483  33318.569              -1.324
       16       7473.277   7867.541              -5.011
       17T18   14669.049  15134.832              -3.078
       19      32556.805  36977.334             -11.955
       20T21   24556.514  27134.519              -9.501
       22      17612.359  18599.630              -5.308

In [20]:
# Test on household's (HFCE) demand for sector 01T03 in Argentina - split_demand = True
x_new = shock_anything(L, x, Z, Y, split_demand = True, country = 'ARG', sector = '01T03', demand_component = 'HFCE', shock = -0.9)

x_new['before'] = x['indout']
x_new['Relative change, %'] = 100 * (x_new['indout'] - x_new['before'])/ x_new['before']
round(x_new.head(10), 3)

indout     before  Relative change, %
region sector                                          
ARG    01T03   73907.445  88343.425             -16.341
       05T06   30134.937  30539.638              -1.325
       09       4148.218   4231.662              -1.972
       10T12   92875.335  93228.853              -0.379
       13T15   33251.146  33318.569              -0.202
       16       7807.276   7867.541              -0.766
       17T18   15063.634  15134.832              -0.470
       19      36301.628  36977.334              -1.827
       20T21   26740.455  27134.519              -1.452
       22      18448.720  18599.630              -0.811

#### Authors
* **Álvaro Corrales Cano** is a Data Scientist within IBM's Cloud Pak Acceleration team. With a background in Economics, Álvaro specialises in a wide array Econometric techniques and causal inference, including regression, discrete choice models, time series and duration analysis.
* **Deepak Shankar Srinivasan** is a Developer in R2 Data Labs, Rolls Royce Deutschland, Germany, specializing in Data Science applications for Equipment Health Management and Deep Domain Specific Smart Assistants.




Copyright © IBM Corp. 2020. Licensed under the Apache License, Version 2.0. Released as licensed Sample Materials.
